<a href="https://colab.research.google.com/github/johan-naizu/solar-power-forecasting/blob/main/Solar%20Power%20Generation%20Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, TimeDistributed, Flatten, InputLayer
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

In [ ]:
df=pd.read_csv('https://raw.githubusercontent.com/johan-naizu/dataset/refs/heads/main/dataset4.csv')
df

In [ ]:
df=df.set_index('timestamp_utc')
df=df.rename_axis('DateTime')
df

In [ ]:
print(df.index.dtype)

In [ ]:
df.index = pd.to_datetime(df.index)
df['Seconds']=df.index.map(pd.Timestamp.timestamp)
df

In [ ]:
print(df.index.dtype)

In [ ]:
day = 60*60*24
year = 365.2425 * day

df['Day sin']=np.sin(df['Seconds'] * (2 * np.pi / day))
df['Day cos']=np.cos(df['Seconds'] * (2 * np.pi / day))
df['Year sin']=np.sin(df['Seconds'] * (2 * np.pi / year))
df['Year cos']=np.cos(df['Seconds'] * (2 * np.pi / year))

df

In [ ]:
df=df.drop('Seconds',axis=1)
df

In [ ]:
features = df.loc[:,['temp', 'cloud_cover','shortwave_flux','power','Day sin','Day cos','Year sin','Year cos']]
target = df.loc[:,['power']]

In [ ]:
plt.plot(features['temp'])
plt.xlabel('Time Index')
plt.ylabel('Temperature (C)')
plt.title('Temperature Time Series')

In [ ]:
plt.plot(features['cloud_cover'][:500])
plt.xlabel('Time Index')
plt.ylabel('Cloud Cover')
plt.title('Cloud Cover Time Series(Magnified)')

In [ ]:
plt.plot(features['shortwave_flux'][:100])
plt.xlabel('Time Index')
plt.ylabel('Shortwave Flux (W/m^2)')
plt.title('Shortwave Flux Time Series')

In [ ]:
plt.plot(features['power'][:100])
plt.xlabel('Time Index')
plt.ylabel('Power (W)')
plt.title('Power Time Series')

In [ ]:
plt.plot(df['Day sin'][:100])
plt.xlabel('Time Index')
plt.ylabel('Day Sin')
plt.title('Day Sin Time Series')

In [ ]:
plt.plot(df['Day cos'][:100])
plt.xlabel('Time Index')
plt.ylabel('Day Cos')
plt.title('Day Cos Time Series')

In [ ]:
plt.plot(df['Year sin'])
plt.xlabel('Time Index')
plt.ylabel('Year Sin')
plt.title('Year Sin Time Series')

In [ ]:
plt.plot(df['Year cos'])
plt.xlabel('Time Index')
plt.ylabel('Year Cos')
plt.title('Year Cos Time Series')

In [ ]:
features

In [ ]:
target

In [ ]:
print(features.shape)
print(target.shape)

In [ ]:
features_scaler = MinMaxScaler()
features_scaled = features_scaler.fit_transform(features)
target_scaler = MinMaxScaler()
target_scaled = target_scaler.fit_transform(target)
time_steps=200

In [ ]:
#Converting 2D to 3D (instead of np.reshape)
def Reshape(features,target,time_steps):
  new_features=[]
  new_target=[]
  for i in range(features.shape[0] - time_steps):
    new_features.append([features[j+i] for j in range(time_steps)])
    new_target.append(target[i+time_steps])
  return np.array(new_features),np.array(new_target)
X_data, y_data = Reshape(features_scaled,target_scaled,time_steps)

In [ ]:
print(X_data.shape)
print(y_data.shape)

In [ ]:
test_train_split= int(0.8*X_data.shape[0])
n_features=X_data.shape[2]

In [ ]:
X_train=X_data[:test_train_split]
y_train=y_data[:test_train_split]

X_test=X_data[test_train_split:]
y_test=y_data[test_train_split:]

In [ ]:
print("X_train Shape: ", X_train.shape)
print("y_train Shape: ", y_train.shape)

print("X_test Shape: ", X_test.shape)
print("y_test Shape", y_test.shape)

print("Number of Features: ", n_features)

In [ ]:
subsequences = 10
inner_timesteps = X_train.shape[1]//subsequences

X_train = X_train.reshape((X_train.shape[0], subsequences, inner_timesteps, n_features))
X_test = X_test.reshape((X_test.shape[0], subsequences, inner_timesteps, n_features))

print("X_train Shape: ", X_train.shape)
print("y_train Shape: ", y_train.shape)

print("X_test Shape: ", X_test.shape)
print("y_test Shape", y_test.shape)

print("Number of Features: ", n_features)

In [ ]:
model1=Sequential()
model1.add(InputLayer((subsequences,inner_timesteps,n_features)))
model1.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
model1.add(TimeDistributed(Flatten()))
model1.add(LSTM(128, activation='tanh',return_sequences=False))
model1.add(Dense(64, activation='relu'))
model1.add(Dropout(0.3))
model1.add(Dense(1 , activation='linear',kernel_regularizer=l2(0.02)))

In [ ]:
cp=ModelCheckpoint('model1.keras', monitor='val_loss', save_best_only=True)
es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

In [ ]:
model1.summary()

In [ ]:
history = model1.fit(X_train, y_train, epochs=100, callbacks=[cp,es], validation_split=0.2)

In [ ]:
train_loss = history.history['loss']  # Training loss
val_loss = history.history.get('val_loss')  # Validation loss

# Plotting the loss
epochs = range(1, len(train_loss) + 1)  # Epoch numbers

plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss, label='Training Loss', color='b', marker='o')
if val_loss:
    plt.plot(epochs, val_loss, label='Validation Loss', color='g', marker='x')
plt.xlabel('Epochs')
plt.ylabel('Loss (Mean Squared Error)')
plt.title('Loss Variation with Epochs')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
from tensorflow.keras.models import load_model
model1=load_model('model1.keras')

In [ ]:
#Evaluating Overfitting
train_predictions = model1.predict(X_train)
train_predictions = target_scaler.inverse_transform(train_predictions).flatten()
train_actuals = target_scaler.inverse_transform(y_train).flatten()
train_results=pd.DataFrame(data={'Predictions':train_predictions,'Actuals':train_actuals})
train_results

In [ ]:
print(train_results.shape)

In [ ]:
plt.plot(train_results['Predictions'][:100],label='Predicted')
plt.plot(train_results['Actuals'][:100],label='Actual')
plt.grid(alpha=0.3)
plt.legend()
plt.xlabel('Time Index')
plt.ylabel('Generated Power (W)')
plt.title("Temporal Prediction Visualization (Training Data)")
plt.show()

In [ ]:
rmse_train = np.sqrt(mean_squared_error(train_results['Predictions'], train_results['Actuals']))
mae_train = mean_absolute_error(train_results['Actuals'],train_results['Predictions'])
mse_train = mean_squared_error(train_results['Actuals'],train_results['Predictions'])
mape_train = mean_absolute_percentage_error(train_results['Actuals'],train_results['Predictions'])
r2_train = r2_score(train_results['Actuals'],train_results['Predictions'])

In [ ]:
print("Mean Absolute Error = ", mae_train)
print("Mean Squared Error = ", mse_train)
print("Root Mean Squared Error =", rmse_train)
print("Mean Absolute Percentage Error =", mape_train)
print("r2 Score =", r2_train)

In [ ]:
#Test data
test_predictions = model1.predict(X_test)
test_predictions = target_scaler.inverse_transform(test_predictions).flatten()
test_actuals = target_scaler.inverse_transform(y_test).flatten()
test_results=pd.DataFrame(data={'Predictions':test_predictions,'Actuals':test_actuals})
test_results

In [ ]:
print(test_results.shape)

In [ ]:
plt.plot(test_results['Predictions'][:100],label='Predicted')
plt.plot(test_results['Actuals'][:100],label='Actual')
plt.grid(alpha=0.3)
plt.legend()
plt.xlabel('Time Index')
plt.ylabel('Generated Power (W)')
plt.title("Temporal Prediction Visualization (Testing Data)")
plt.show()

In [ ]:
rmse_test = np.sqrt(mean_squared_error(test_results['Predictions'], test_results['Actuals']))
mae_test = mean_absolute_error(test_results['Actuals'],test_results['Predictions'])
mse_test = mean_squared_error(test_results['Actuals'],test_results['Predictions'])
mape_test = mean_absolute_percentage_error(test_results['Actuals'],test_results['Predictions'])
r2_test = r2_score(test_results['Actuals'],test_results['Predictions'])

In [ ]:
print("Mean Absolute Error = ", mae_test)
print("Mean Squared Error = ", mse_test)
print("Root Mean Squared Error =", rmse_test)
print("Mean Absolute Percentage Error =", mape_test)
print("r2 Score =", r2_test)

In [ ]:
residual_test=pd.DataFrame(data={'Residuals':test_results['Predictions']-test_results['Actuals']})
plt.plot(residual_test['Residuals'],label='Residuals')
plt.axhline(y=0, color='r', linestyle='-')
plt.grid(alpha=0.3)
plt.legend()
plt.xlabel('Time Index')
plt.ylabel('Residuals')
plt.title("Residual Plot Magnified(Predictions - Actuals)")
plt.show()

In [ ]:
# Metrics for train and test data
metrics = ['RMSE', 'MAE', 'R² score * 1000']
train_scores = [rmse_train, mae_train , r2_train*1000]
test_scores = [rmse_test,mae_test,r2_test*1000]
scores_df = pd.DataFrame({'Metrics': metrics, 'Train Scores': train_scores, 'Test Scores': test_scores})

# Positions for bar groups
x = np.arange(len(metrics))
width = 0.35

# Create the plot
plt.bar(x - width/2, scores_df['Train Scores'], width, label='Train', color='b', alpha=0.7)
plt.bar(x + width/2, scores_df['Test Scores'], width, label='Test', color='g', alpha=0.7)

for i, v in enumerate(scores_df['Train Scores']):
    plt.text(x[i] - width/2, v + 0.02, f'{v:.2f}', ha='center', va='bottom', fontsize=10)

for i, v in enumerate(scores_df['Test Scores']):
    plt.text(x[i] + width/2, v + 0.02, f'{v:.2f}', ha='center', va='bottom', fontsize=10)

# Add labels, title, and legend
plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Train vs Test Metrics Comparison')
plt.xticks(x, scores_df['Metrics'])  # Add metric names as x-axis labels
plt.legend()
plt.grid(alpha=0.3)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
plt.hist(residual_test['Residuals'], bins=30, edgecolor='k', alpha=0.7)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals')
plt.grid(alpha=0.3)
plt.show()

In [ ]:
plt.scatter(test_results['Actuals'], test_results['Predictions'], alpha=0.6, edgecolor='k')
plt.plot([min(test_results['Actuals']), max(test_results['Actuals'])],
         [min(test_results['Actuals']), max(test_results['Actuals'])],
         color='r', linestyle='--', label='Ideal Prediction')
plt.xlabel('Actual Values (W)')
plt.ylabel('Predicted Values (W)')
plt.title('Predictions vs Actuals')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
residual_cumsum = residual_test['Residuals'].cumsum()
plt.plot(residual_cumsum, color='purple')
plt.xlabel('Time Index')
plt.ylabel('Cumulative Error')
plt.title('Cumulative Prediction Error Over Time')
plt.grid(alpha=0.3)
plt.show()

In [ ]:
plot_model(model1, show_shapes=True, show_layer_names=True, to_file='model_architecture.png')